In [1]:
import pandas as pd

In [2]:
loan_data = pd.read_csv('datasets/loan_small.csv')
display(loan_data) # Display the data


,Loan_ID,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Area,Loan_Status
0,LP001002,NaN,5849.0,0.0,NaN,urban,Y
1,LP001003,Male,4583.0,NaN,128.0,semi,N
2,LP001005,Male,3000.0,0.0,66.0,NaN,Y
3,LP001006,Female,2583.0,2358.0,120.0,semi,NaN
4,LP001008,Male,NaN,0.0,141.0,urban,Y
5,LP001011,Male,5417.0,4196.0,267.0,semi,Y
6,LP001013,Male,2333.0,1516.0,NaN,rural,Y
7,LP001014,Female,3036.0,2504.0,158.0,semi,N
8,LP001018,Male,4006.0,1526.0,168.0,rural,Y
9,LP001020,Male,12841.0,10968.0,349.0,semi,N


In [3]:
# Check for exact duplicated rows in the data 
loan_data.duplicated().sum()

np.int64(0)

In [4]:
# Check for duplicated rows in case there are any
# Note - It will return only the duplicated rows from the entire dataset
loan_data.loc[loan_data.duplicated(), :]

,Loan_ID,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Area,Loan_Status


In [5]:
# Dropping duplicate rows if any by keeping only 'first' or 'last'
loan_data.drop_duplicates(keep='first', inplace=True)

In [6]:
Q1 = loan_data.select_dtypes(include='number').quantile(0.25, axis = 0) # 25% of the data

print('First_Quartile :\n', Q1)

First_Quartile :
 ApplicantIncome      2520.75
CoapplicantIncome     350.00
LoanAmount            109.00
Name: 0.25, dtype: float64


In [7]:
Q3 = loan_data.select_dtypes(include='number').quantile(0.75, axis = 0) # 75% of the data

print('Third_Quartile :\n', Q3)

Third_Quartile :
 ApplicantIncome      4858.25
CoapplicantIncome    2672.00
LoanAmount            158.00
Name: 0.75, dtype: float64


The 25th(Q1) and 75th(Q3) quartiles are obtained and Inter quartile range(IQR) is obtained from Q3 and Q1

In [8]:
IQR = Q3 - Q1

print('Inter Quartile Range :\n', IQR)

Inter Quartile Range :
 ApplicantIncome      2337.5
CoapplicantIncome    2322.0
LoanAmount             49.0
dtype: float64


Outliers are detected using IQR method i.e An outlier is a point which falls more than 1.5 times the interquartile range above the third quartile or below the first quartile.


In [9]:
# Only the outlier rows are displayed 
loan_data_outliers = loan_data[
    (
        (loan_data.select_dtypes(include='number') < (Q1 - 1.5 * IQR)) 
        |
        (loan_data.select_dtypes(include='number') > (Q3 + 1.5 * IQR))
    ).any(axis = 1)
]

loan_data_outliers

,Loan_ID,Gender,ApplicantIncome,CoapplicantIncome,LoanAmount,Area,Loan_Status
5,LP001011,Male,5417.0,4196.0,267.0,semi,Y
9,LP001020,Male,12841.0,10968.0,349.0,semi,N
12,LP001028,Female,NaN,8106.0,NaN,urban,Y
14,LP001030,Male,1299.0,1086.0,17.0,semi,Y


In [10]:
print(f"Initial shape: {loan_data.shape}")

Initial shape: (16, 7)


The final dataframe is obtained after removing the outlier rows

In [11]:
loan_data = loan_data[
    ~(
        (loan_data.select_dtypes('number') < (Q1 - 1.5 * IQR)) 
        |
        (loan_data.select_dtypes('number') > (Q3 + 1.5 * IQR))
    ).any(axis = 1)
]

print(f"Shape after removing outliers: {loan_data.shape}")

Shape after removing outliers: (12, 7)


In [12]:
loan_data.isnull().sum() # Check for missing values

Loan_ID              0
Gender               1
ApplicantIncome      1
CoapplicantIncome    1
LoanAmount           2
Area                 1
Loan_Status          1
dtype: int64

For numbers - Either remove records with null values or substitute them using mean of the column. Since its a small dataset, we can go ahead with substituting with mean value of the column.

In [13]:
loan_data.select_dtypes('number').fillna(loan_data.select_dtypes('number').mean()) 
# Fill missing values with the mean

,ApplicantIncome,CoapplicantIncome,LoanAmount
0,5849.000000,0.000000,119.9
1,4583.000000,1207.636364,128.0
2,3000.000000,0.000000,66.0
3,2583.000000,2358.000000,120.0
4,3444.818182,0.000000,141.0
6,2333.000000,1516.000000,119.9
7,3036.000000,2504.000000,158.0
8,4006.000000,1526.000000,168.0
10,3200.000000,700.000000,70.0
11,2500.000000,1840.000000,109.0


Loan_ID is not of any importance, so we drop it

In [14]:
loan_data = loan_data.drop(['Loan_ID'], axis=1)

For categorical columns<br>

Option 1- Convert to dummy variables. `pandas.get_dummies()` is a function in the Pandas library that converts categorical data (variables with a fixed number of distinct categories) into a format that can be provided to machine learning models. This process is called one-hot encoding. It takes a column with categorical values (e.g., ["Male", "Female"]) and creates separate columns for each unique category (e.g., Male, Female), with binary values (1/True or 0/False) indicating the presence of that category in the data.<br>

Option 2 - Use most frequently occuring value to fill i.e. mode()<br>

Here will go ahead with option 1<br>


In [15]:
for_temporary_observation = pd.get_dummies(loan_data)
print(for_temporary_observation) # Display the data

    ApplicantIncome  CoapplicantIncome  LoanAmount  Gender_Female  \
0            5849.0                0.0         NaN          False   
1            4583.0                NaN       128.0          False   
2            3000.0                0.0        66.0          False   
3            2583.0             2358.0       120.0           True   
4               NaN                0.0       141.0          False   
6            2333.0             1516.0         NaN          False   
7            3036.0             2504.0       158.0           True   
8            4006.0             1526.0       168.0          False   
10           3200.0              700.0        70.0           True   
11           2500.0             1840.0       109.0          False   
13           1853.0             2840.0       114.0          False   
15           4950.0                0.0       125.0          False   

    Gender_Male  Area_rural  Area_semi  Area_urban  Loan_Status_N  \
0         False       False      

If you're using one-hot encoding in regression or other linear models, you can set drop_first=True to avoid multicollinearity (avoiding redundancy)

In [16]:
for_temporary_observation = pd.get_dummies(loan_data, drop_first=True)
print(for_temporary_observation) # Display the data

    ApplicantIncome  CoapplicantIncome  LoanAmount  Gender_Male  Area_semi  \
0            5849.0                0.0         NaN        False      False   
1            4583.0                NaN       128.0         True       True   
2            3000.0                0.0        66.0         True      False   
3            2583.0             2358.0       120.0        False       True   
4               NaN                0.0       141.0         True      False   
6            2333.0             1516.0         NaN         True      False   
7            3036.0             2504.0       158.0        False       True   
8            4006.0             1526.0       168.0         True      False   
10           3200.0              700.0        70.0        False      False   
11           2500.0             1840.0       109.0         True      False   
13           1853.0             2840.0       114.0         True      False   
15           4950.0                0.0       125.0         True 

As you notice, even if we have used `drop_first=True` to avoid redundancy, both the Gender_Male and Gender_Female columns still exists (only one should be present). Similarly, for Loan_Status, Area

`pd.get_dummies` may not have correctly recognized it as a categorical column. Instead, it could have treated it as a set of independent binary columns (Gender_Female and Gender_Male), preventing one from being dropped.

In [17]:
loan_data['Gender'] = loan_data['Gender'].astype('category')
loan_data['Loan_Status'] = loan_data['Loan_Status'].astype('category')
loan_data['Area'] = loan_data['Area'].astype('category')

In [18]:
# Example usage of get_dummies only on the 'Gender' column
only_for_gender = pd.get_dummies(loan_data, columns=['Gender'], prefix='new')
only_for_gender

,ApplicantIncome,CoapplicantIncome,LoanAmount,Area,Loan_Status,new_Female,new_Male
0,5849.0,0.0,NaN,urban,Y,False,False
1,4583.0,NaN,128.0,semi,N,False,True
2,3000.0,0.0,66.0,NaN,Y,False,True
3,2583.0,2358.0,120.0,semi,NaN,True,False
4,NaN,0.0,141.0,urban,Y,False,True
6,2333.0,1516.0,NaN,rural,Y,False,True
7,3036.0,2504.0,158.0,semi,N,True,False
8,4006.0,1526.0,168.0,rural,Y,False,True
10,3200.0,700.0,70.0,urban,Y,True,False
11,2500.0,1840.0,109.0,urban,Y,False,True


In [19]:
loan_data = pd.get_dummies(loan_data, drop_first=True)
print(loan_data)

    ApplicantIncome  CoapplicantIncome  LoanAmount  Gender_Male  Area_semi  \
0            5849.0                0.0         NaN        False      False   
1            4583.0                NaN       128.0         True       True   
2            3000.0                0.0        66.0         True      False   
3            2583.0             2358.0       120.0        False       True   
4               NaN                0.0       141.0         True      False   
6            2333.0             1516.0         NaN         True      False   
7            3036.0             2504.0       158.0        False       True   
8            4006.0             1526.0       168.0         True      False   
10           3200.0              700.0        70.0        False      False   
11           2500.0             1840.0       109.0         True      False   
13           1853.0             2840.0       114.0         True      False   
15           4950.0                0.0       125.0         True 

In [20]:
loan_data.dropna(inplace=True) # Drop missing values

In [21]:
display(loan_data) # Display the data

,ApplicantIncome,CoapplicantIncome,LoanAmount,Gender_Male,Area_semi,Area_urban,Loan_Status_Y
2,3000.0,0.0,66.0,True,False,False,True
3,2583.0,2358.0,120.0,False,True,False,False
7,3036.0,2504.0,158.0,False,True,False,False
8,4006.0,1526.0,168.0,True,False,False,True
10,3200.0,700.0,70.0,False,False,True,True
11,2500.0,1840.0,109.0,True,False,True,True
13,1853.0,2840.0,114.0,True,False,True,False
15,4950.0,0.0,125.0,True,True,False,True


In [22]:
loan_data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount
count,8.000000,8.000000,8.000000
mean,3141.000000,1471.000000,116.250000
std,958.889387,1120.543746,36.319023
min,1853.000000,0.000000,66.000000
25%,2562.250000,525.000000,99.250000
50%,3018.000000,1683.000000,117.000000
75%,3401.500000,2394.500000,133.250000
max,4950.000000,2840.000000,168.000000
